<a href="https://colab.research.google.com/github/sttutie/Jungho-Shin/blob/main/quickstart_tutorial_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colab에서 노트북을 실행하실 때에는
# https://tutorials.pytorch.kr/beginner/colab 를 참고하세요.
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# 학습 데이터를 내려받고 변수 이름을 kimchi로
kimchi = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),  # 이미지를 Tensor로 변환
)

# 테스트 데이터를 내려받고 변수 이름을 bokkumbob으로 한다.
bokkumbob = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),  # 이미지를 Tensor로 변환
)

100%|██████████| 26421880/26421880 [00:01<00:00, 20728510.80it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 515281.70it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6308455.05it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12761393.02it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
batch_size = 64

#이곳변형 김치로더와 볶음밥로더를 사용한다.
kimchi_loader = DataLoader(kimchi, batch_size=batch_size)#트레이닝
bokkumbob_loader = DataLoader(bokkumbob, batch_size=batch_size)#테스트

for X, y in bokkumbob_loader:
    print(f"X의 형태 [N, C, H, W]: {X.shape}")#이곳변형 한국어를 썼다. 신토불이.
    print(f"y의 형태: {y.shape} {y.dtype}")
    break

X의 형태 [N, C, H, W]: torch.Size([64, 1, 28, 28])
y의 형태: torch.Size([64]) torch.int64


In [6]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),  #이곳변형 시그모이드 활성화 함수 사용
            nn.Linear(512, 512),
            nn.Sigmoid(),  #이곳변형 시그모이드 활성화 함수 사용
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
kamja_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, kamja_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        kamja = kamja_fn(pred, y)#이곳변형 감자만 먹으면 영양이 loss 납니다. 그래서 loss 이름은 감자로 했습니다.

        # 역전파
        optimizer.zero_grad()
        kamja.backward()
        optimizer.step()

        if batch % 100 == 0:
            kamja, current = kamja.item(), (batch + 1) * len(X)#이곳변형 변수에 맞춰 변경
            print(f"kamja: {kamja:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, kamja_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_kamja, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_kamja += kamja_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_kamja /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg kamja: {test_kamja:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(kimchi_loader, model, kamja_fn, optimizer)
    test(bokkumbob_loader, model, kamja_fn)
print("Done!")

Epoch 1
-------------------------------
kamja: 2.353183  [   64/60000]
kamja: 2.314624  [ 6464/60000]
kamja: 2.300983  [12864/60000]
kamja: 2.304245  [19264/60000]
kamja: 2.301820  [25664/60000]
kamja: 2.305883  [32064/60000]
kamja: 2.303224  [38464/60000]
kamja: 2.300146  [44864/60000]
kamja: 2.301665  [51264/60000]
kamja: 2.305307  [57664/60000]
Test Error: 
 Accuracy: 10.0%, Avg kamja: 2.300322 

Epoch 2
-------------------------------
kamja: 2.300128  [   64/60000]
kamja: 2.304006  [ 6464/60000]
kamja: 2.297414  [12864/60000]
kamja: 2.302290  [19264/60000]
kamja: 2.299113  [25664/60000]
kamja: 2.303485  [32064/60000]
kamja: 2.300795  [38464/60000]
kamja: 2.297745  [44864/60000]
kamja: 2.299260  [51264/60000]
kamja: 2.302773  [57664/60000]
Test Error: 
 Accuracy: 10.0%, Avg kamja: 2.297888 

Epoch 3
-------------------------------
kamja: 2.297761  [   64/60000]
kamja: 2.301816  [ 6464/60000]
kamja: 2.294883  [12864/60000]
kamja: 2.300213  [19264/60000]
kamja: 2.296443  [25664/60000]

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = bokkumbob[0][0], bokkumbob[0][1]#이곳변형 볶음밥은 최고다
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Coat", Actual: "Ankle boot"
